# Code handout - Assignment 4 - Satellite

In this assignment, you will write a small simulator for a satellite in orbit.
In order to simulate and render the satellite, you will need to write some code.
However, much of the code is already written for you (mostly to make things compatible with the visualization).

The only code you need to understand is the code under **Satellite dynamics** and **Simulation**,
the animation code in **Animation** is only for visualization.
Sadly, the visualization only seems to work in the web browser, so if you are running the notebooks in (for instance) vs-code, you will either need a different method for figuring out the difference between the cases or you will need to run a jupyter-server or something similar.

Good luck!

## Imports

For this assingment we will need:
- NumPy for vectors/matrices
- SciPy for solving the ODEs and for rotation matrices

You may want to add more imports, but it should be necessary to do everything with this.

In [2]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.spatial.transform import Rotation

## Satellite dynamics

In order to simulate the dynamics of the satellite, we need the ODEs for the state variables.
The `make_satellite_dynamics`-function will return exactly this.

You will have to fill in the expressions that calculate the time derivative of:

- Position: $r_c^i$, `i_r_c`
- Rotation: $q$, `q` (as a quaternion)
- Velocity: $v_c^i$, `i_v_c`
- Angular velocity: $\omega_{i/b}^b$, `b_omega_ib`

In [3]:
def skew(v):
    return np.array([[0, -v[2], v[1]],
                     [v[2], 0, -v[0]],
                     [-v[1], v[0], 0]])

def q_derivative(q, omega):
    qw, qx, qy, qz = q
    wx, wy, wz = omega
    q_dot = 0.5 * np.array([
        - qx*wx - qy*wy - qz*wz,
         qw*wx + qy*wz - qz*wy,
         qw*wy - qx*wz + qz*wx,
         qw*wz + qx*wy - qy*wx
    ])
    return q_dot

def make_satellite_dynamics(b_M_c, G, m_T):
    """Wraps `satellite_dynamics` in a scope with necessary constants defined"""
    def satellite_dynamics(t, x):
        i_r_c = x[0:3]
        q = x[3:7]
        i_v_c = x[7:10]
        b_omega_ib = x[10:13]

        # Kinematics
        i_r_c_dot = i_v_c.copy()       # TODO
        q_dot = q_derivative(q, b_omega_ib)           # TODO

        r_norm = np.linalg.norm(i_r_c)
        
        # Dynamics
        i_v_c_dot = - (G * m_T / r_norm**3) * i_r_c       # TODO
        b_omega_ib_dot = - np.linalg.inv(b_M_c) @ np.cross(b_omega_ib, b_M_c @ b_omega_ib)  # TODO

        state_dot = np.zeros_like(x)
        state_dot[0:3] = i_r_c_dot[:]
        state_dot[3:7] = q_dot[:]
        state_dot[7:10] = i_v_c_dot[:]
        state_dot[10:13] = b_omega_ib_dot[:]

        

        return state_dot

    return satellite_dynamics

## Simulation

Now, in order to simulate the trajectory, you will need to provide the correct expressions
for the inertia matrices around the relevant mass centers.
There are two cases; one without the added mass and one with!

In the first case, you already have all you need.
In the second case, you will need:

- $r_s^b$, `b_r_s`
    - Displacement from mass center to satellite center
- $M_o^b$, `b_M_o`
    - Inertia matrix with respect to the old center of mass
- $M_c^b$, `b_M_c`
    - Inertia matrix with respect to the (new) center of mass

All vectors and matrices are in the body frame which has its origin in the center of the satellite.


In [4]:
# Constants
earth_radius = 6356e+3
orbit_height = 36000e+3
azi = np.pi / 4
dec = np.pi / 4

# Initial state
pos = (earth_radius + orbit_height) * \
    np.array([np.sin(dec) * np.cos(azi), np.sin(dec) * np.sin(azi), np.cos(dec)])
q = Rotation.random().as_quat()
vel = np.zeros((3,))
ang_vel = np.array([np.deg2rad(60), np.deg2rad(80), np.deg2rad(100)])
state = np.hstack((pos, q, vel, ang_vel))

# Parameters
m_T = 5.972e+24
G = 6.67e-11
l = 0.5
rho = 1
m = rho * l**3

with_added_mass = False

if with_added_mass:
    m_added = 0.1
    total_mass = m + m_added
    b_r_0 = l * np.ones((3,))  # Location of the added mass relative to satellite center
    b_r_s = - (m_added / total_mass) * b_r_0     # TODO
    I_cube = (1/6) * m * (l**2) * np.eye(3)
    I_added = m_added * (np.dot(b_r_0, b_r_0) * np.eye(3) - np.outer(b_r_0, b_r_0))
    b_M_o = I_cube + I_added   # TODO
    b_M_c = b_M_o + total_mass * (np.dot(b_r_s, b_r_s) * np.eye(3) - np.outer(b_r_s, b_r_s))    # TODO
else:
    total_mass = m
    b_M_c = (m * (l**2) / 6) * np.eye(3)  # Interia matrix around the center of mass
    b_r_s = np.zeros((3,))                # Displacement of satellite center from mass center
    

pos = (earth_radius + orbit_height) * np.array([
    np.sin(dec) * np.cos(azi),
    np.sin(dec) * np.sin(azi),
    np.cos(dec)
])

q = Rotation.random().as_quat()  
q = np.array([q[3], q[0], q[1], q[2]])

vel = np.zeros((3,))

ang_vel = np.deg2rad(np.array([60, 80, 100]))

state = np.hstack((pos, q, vel, ang_vel))

satellite_dynamics = make_satellite_dynamics(b_M_c, G, m_T)
time_final = 12

result = solve_ivp(satellite_dynamics, (0, time_final), state, 
                    t_eval = np.linspace(0, time_final, num=1001))

time = result.t
state_trajectory = result.y
positions = (1e-7 * state_trajectory[0:3]).T  # Scaled to produce positions that are not literally thousands of miles away!
rotations = state_trajectory[3:7].T

## Animation

We try to animate the satellite using the Python-library `pythreejs`, which is a wrapper library for `three.js`.
It is *not necessary* to understand the animation code, it should just work.
Provided one is running the notebook in the web browser through a jupyter-server (e.g. not in vs-code).
If the animation seems to shake, contract or "shudder", try changing the number of simulated timesteps.

In [1]:
import pythreejs as pj
from IPython.display import display

# Paths to the cubemap images (6 faces)
folder = "space_cubemap"
texture_paths = [
    f"./{folder}/px.png",  # Positive X
    f"./{folder}/nx.png",  # Negative X
    f"./{folder}/py.png",  # Positive Y
    f"./{folder}/ny.png",  # Negative Y
    f"./{folder}/pz.png",  # Positive Z
    f"./{folder}/nz.png",  # Negative Z
]

box_sides = 500
scene = pj.Scene()

camera = pj.PerspectiveCamera(position=[0, 0, 5], up=[0, 1, 0], aspect=1)
camera.lookAt([0, 0, 0])

# Create a CubeGeometry to represent the skybox
geometry = pj.BoxGeometry(width=box_sides, height=box_sides, depth=box_sides)

materials = []
for path in texture_paths:
    texture = pj.ImageTexture(imageUri=path)
    material = pj.MeshBasicMaterial(map=texture, side='BackSide')
    materials.append(material)

skybox = pj.Mesh(geometry, materials)
scene.add(skybox)

cubesat = pj.Mesh(
    pj.BoxGeometry(1, 1, 1),
    pj.MeshStandardMaterial(color='red')
)
the_added_mass = pj.Mesh(
    pj.BoxGeometry(0.1, 0.1, 0.1),
    pj.MeshStandardMaterial(color="green")
)

cubesat.position = tuple(b_r_s)
the_added_mass.position = tuple(b_r_s + .5*np.ones((3,)))

axis_helper = pj.AxesHelper(size=5)
axis_helper.position = cubesat.position
axis_helper.quaternion = cubesat.quaternion

pivot = pj.Group()
pivot.add(cubesat)
if with_added_mass: pivot.add(the_added_mass)
pivot.add(axis_helper)

pivot_position_track = pj.VectorKeyframeTrack(name=".position", times = time, values = positions)
pivot_rotation_track = pj.QuaternionKeyframeTrack(name = ".quaternion", times = time, values = rotations)
pivot_clip = pj.AnimationClip(tracks = [pivot_position_track, pivot_rotation_track])
pivot_action = pj.AnimationAction(pj.AnimationMixer(pivot), pivot_clip, pivot)
scene.add(pivot)

view_width = 800
view_height = 600
camera = pj.PerspectiveCamera(position=[10, 6, 10], aspect=view_width/view_height)

ambient_light = pj.AmbientLight(color='#ffffff', intensity=1.0)
key_light = pj.DirectionalLight(position=[0, 10, 0])
scene.add(key_light)
scene.add(ambient_light)

renderer = pj.Renderer(camera=camera, scene=scene, width=view_width, height=view_height)
controls = pj.OrbitControls(controlling=camera)
renderer.controls = [controls]

display(renderer)
pivot_action

ModuleNotFoundError: No module named 'pythreejs'